In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import argparse
import xml.etree.ElementTree as ET
import math
from tensorflow.keras import regularizers
from tensorflow.keras.layers import BatchNormalization, GaussianNoise

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/DataSets/car_img-test.zip" -d /content/car_image_test

Streaming output truncated to the last 5000 lines.
  inflating: /content/car_image_test/test/day_08471.jpg  
  inflating: /content/car_image_test/test/day_08471.xml  
  inflating: /content/car_image_test/test/day_08473.jpg  
  inflating: /content/car_image_test/test/day_08473.xml  
  inflating: /content/car_image_test/test/day_08475.jpg  
  inflating: /content/car_image_test/test/day_08475.xml  
  inflating: /content/car_image_test/test/day_08477.jpg  
  inflating: /content/car_image_test/test/day_08477.xml  
  inflating: /content/car_image_test/test/day_08478.jpg  
  inflating: /content/car_image_test/test/day_08478.xml  
  inflating: /content/car_image_test/test/day_08487.jpg  
  inflating: /content/car_image_test/test/day_08487.xml  
  inflating: /content/car_image_test/test/day_08493.jpg  
  inflating: /content/car_image_test/test/day_08493.xml  
  inflating: /content/car_image_test/test/day_08494.jpg  
  inflating: /content/car_image_test/test/day_08494.xml  
  inflating: /content

In [ ]:
!unzip "/content/drive/MyDrive/DataSets/car_img-validation.zip" -d /content/car_image_validation

Archive:  /content/drive/MyDrive/DataSets/car_img-validation.zip
   creating: /content/car_image_validation/validation/
  inflating: /content/car_image_validation/validation/day_00004.jpg  
  inflating: /content/car_image_validation/validation/day_00004.xml  
  inflating: /content/car_image_validation/validation/day_00011.jpg  
  inflating: /content/car_image_validation/validation/day_00011.xml  
  inflating: /content/car_image_validation/validation/day_00018.jpg  
  inflating: /content/car_image_validation/validation/day_00018.xml  
  inflating: /content/car_image_validation/validation/day_00024.jpg  
  inflating: /content/car_image_validation/validation/day_00024.xml  
  inflating: /content/car_image_validation/validation/day_00033.jpg  
  inflating: /content/car_image_validation/validation/day_00033.xml  
  inflating: /content/car_image_validation/validation/day_00036.jpg  
  inflating: /content/car_image_validation/validation/day_00036.xml  
  inflating: /content/car_image_validati

In [ ]:
!unzip "/content/drive/MyDrive/DataSets/plate_image_with_dummy-train.zip" -d /content/plate_image_train

Streaming output truncated to the last 5000 lines.
  inflating: /content/plate_image_train/train/dummy_te_14501.jpg  
  inflating: /content/plate_image_train/train/dummy_te_14501.xml  
  inflating: /content/plate_image_train/train/dummy_te_14502.jpg  
  inflating: /content/plate_image_train/train/dummy_te_14502.xml  
  inflating: /content/plate_image_train/train/dummy_te_14503.jpg  
  inflating: /content/plate_image_train/train/dummy_te_14503.xml  
  inflating: /content/plate_image_train/train/dummy_te_14504.jpg  
  inflating: /content/plate_image_train/train/dummy_te_14504.xml  
  inflating: /content/plate_image_train/train/dummy_te_14505.jpg  
  inflating: /content/plate_image_train/train/dummy_te_14505.xml  
  inflating: /content/plate_image_train/train/dummy_te_14506.jpg  
  inflating: /content/plate_image_train/train/dummy_te_14506.xml  
  inflating: /content/plate_image_train/train/dummy_te_14507.jpg  
  inflating: /content/plate_image_train/train/dummy_te_14507.xml  
  inflating

In [ ]:
import os, glob, random, yaml, cv2
import xml.etree.ElementTree as ET

# -----------------------------
# Step 1: Gather Unique Characters
# -----------------------------
plate_dir = "/content/plate_image_train/train"  # Folder with your 45K plate images and XML files

# Scan through all XML files to extract unique characters
xml_files = glob.glob(os.path.join(plate_dir, "*.xml"))
unique_chars = set()
for xml_file in xml_files:
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for obj in root.findall("object"):
            char = obj.find("name").text.strip()
            unique_chars.add(char)
    except Exception as e:
        print(f"Error parsing {xml_file}: {e}")

unique_chars = sorted(unique_chars)
print("Unique characters found:", unique_chars)

Unique characters found: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'D', 'S', 'الف', 'ب', 'ت', 'تشریفات', 'ث', 'ج', 'د', 'ز', 'س', 'ش', 'ص', 'ط', 'ظ', 'ع', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'ه\u200d', 'و', 'پ', 'ژ (معلولین و جانبازان)', 'ک', 'گ', 'ی']


In [ ]:
# Create mapping from character to index
char_to_idx = {char: idx for idx, char in enumerate(unique_chars)}
print("Character mapping:", char_to_idx)

# -----------------------------
# Step 2: Convert XML to YOLO-format Labels
# -----------------------------
def convert_plate_xml_to_yolo(xml_file, img_width, img_height):
    """
    Converts a plate XML file to YOLO-format labels for individual characters.
    Each line: <class> <x_center> <y_center> <width> <height> (all normalized)
    """
    tree = ET.parse(xml_file)
    root = tree.getroot()
    lines = []
    for obj in root.findall("object"):
        char = obj.find("name").text.strip()
        cls_idx = char_to_idx[char]
        bndbox = obj.find("bndbox")
        xmin = float(bndbox.find("xmin").text)
        ymin = float(bndbox.find("ymin").text)
        xmax = float(bndbox.find("xmax").text)
        ymax = float(bndbox.find("ymax").text)
        # Convert to normalized center coordinates
        x_center = ((xmin + xmax) / 2.0) / img_width
        y_center = ((ymin + ymax) / 2.0) / img_height
        box_width = (xmax - xmin) / img_width
        box_height = (ymax - ymin) / img_height
        lines.append(f"{cls_idx} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}")
    return lines

# Process each XML file and write a corresponding .txt label file
converted_count = 0
for xml_file in xml_files:
    base = os.path.splitext(xml_file)[0]
    image_file = base + ".jpg"
    if not os.path.exists(image_file):
        continue
    img = cv2.imread(image_file)
    if img is None:
        continue
    h, w = img.shape[:2]
    yolo_lines = convert_plate_xml_to_yolo(xml_file, w, h)
    if yolo_lines:
        with open(base + ".txt", "w", encoding="utf-8") as f:
            f.write("\n".join(yolo_lines))
        converted_count += 1
print(f"Converted {converted_count} XML files to YOLO-format labels.")

Character mapping: {'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, 'D': 10, 'S': 11, 'الف': 12, 'ب': 13, 'ت': 14, 'تشریفات': 15, 'ث': 16, 'ج': 17, 'د': 18, 'ز': 19, 'س': 20, 'ش': 21, 'ص': 22, 'ط': 23, 'ظ': 24, 'ع': 25, 'ف': 26, 'ق': 27, 'ل': 28, 'م': 29, 'ن': 30, 'ه': 31, 'ه\u200d': 32, 'و': 33, 'پ': 34, 'ژ (معلولین و جانبازان)': 35, 'ک': 36, 'گ': 37, 'ی': 38}
Converted 45469 XML files to YOLO-format labels.


In [ ]:
# -----------------------------
# Step 3: Split Dataset into Train/Val/Test
# -----------------------------
# Gather all plate images (assume .jpg files in plate_dir)
all_imgs = glob.glob(os.path.join(plate_dir, "*.jpg"))
all_basenames = [os.path.splitext(os.path.basename(f))[0] for f in all_imgs]

random.shuffle(all_basenames)
n = len(all_basenames)
n_train = int(0.8 * n)
n_val   = int(0.1 * n)
train_set = all_basenames[:n_train]
val_set   = all_basenames[n_train:n_train + n_val]
test_set  = all_basenames[n_train + n_val:]

# Create directories for splits
split_dir = "plate_dataset"
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(split_dir, split), exist_ok=True)

def move_files(basenames, split_name):
    for bn in basenames:
        for ext in [".jpg", ".txt"]:
            src = os.path.join(plate_dir, bn + ext)
            dst = os.path.join(split_dir, split_name, bn + ext)
            if os.path.exists(src):
                os.rename(src, dst)

move_files(train_set, "train")
move_files(val_set, "val")
move_files(test_set, "test")
print("Dataset split into train, val, and test.")

Dataset split into train, val, and test.


In [ ]:
# -----------------------------
# Step 4: Create a YAML File for the Plate Recognition Dataset
# -----------------------------
plate_dataset_yaml = {
    "train": os.path.abspath(os.path.join(split_dir, "train")),
    "val": os.path.abspath(os.path.join(split_dir, "val")),
    "test": os.path.abspath(os.path.join(split_dir, "test")),
    "nc": len(unique_chars),
    "names": unique_chars
}

with open("plate_dataset.yaml", "w", encoding="utf-8") as f:
    yaml.dump(plate_dataset_yaml, f)
print("Plate dataset YAML created:")
print(plate_dataset_yaml)

Plate dataset YAML created:
{'train': '/content/plate_dataset/train', 'val': '/content/plate_dataset/val', 'test': '/content/plate_dataset/test', 'nc': 39, 'names': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'D', 'S', 'الف', 'ب', 'ت', 'تشریفات', 'ث', 'ج', 'د', 'ز', 'س', 'ش', 'ص', 'ط', 'ظ', 'ع', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'ه\u200d', 'و', 'پ', 'ژ (معلولین و جانبازان)', 'ک', 'گ', 'ی']}


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.8/921.8 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [ ]:
# -----------------------------
# Step 5: Train the YOLO Model for Plate Recognition
# -----------------------------
from ultralytics import YOLO

# Use your preferred model variant; here we use YOLO12n for recognition.
model_plate = YOLO("yolo12n.pt")

results_plate = model_plate.train(
    data="plate_dataset.yaml",   # YAML file for plate recognition
    epochs=50,                   # Adjust epochs as needed
    imgsz=416,
    batch=32,
    #device="cuda",               # Use GPU if available
    augment=True,                # Enable on-the-fly augmentation
    project="runs/plate_recognition",
    name="yolo_plate_recognition",
    cache=True,
    exist_ok=True,
    seed=42,
    workers=12
)

print("Plate recognition training complete. Results:")
print(results_plate)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.34M/5.34M [00:00<00:00, 192MB/s]


Ultralytics 8.3.79 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo12n.pt, data=plate_dataset.yaml, epochs=50, time=None, patience=100, batch=32, imgsz=416, save=True, save_period=-1, cache=True, device=None, workers=12, project=runs/plate_recognition, name=yolo_plate_recognition, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, 

100%|██████████| 22.2M/22.2M [00:00<00:00, 41.0MB/s]


Overriding model.yaml nc=80 with nc=39

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  2    180864  ultralytics.nn.modules.block.A2C2f           [128, 128, 2, True, 4]        
  7                  -1  1    295424  ultralytic

...

      18/50      2.51G     0.2462     0.1975     0.8084        120        416: 100%|██████████| 1137/1137 [05:47<00:00,  3.27it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 71/71 [00:20<00:00,  3.54it/s]

                   all       4544      36036       0.99      0.984      0.992      0.909



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      19/50      2.43G     0.2414     0.1938     0.8078        528        416:  93%|█████████▎| 1054/1137 [05:23<00:25,  3.25it/s]


KeyboardInterrupt: 